# Sistema de Detecção e Reconhecimento de Faces com TensorFlow

**Ambiente utilizado:** Linux Ubuntu 24.04, Python 3.11

**Framework Principal:** TensorFlow

**Componentes:**
1.  **Detector de Faces:** Rede pré-treinada `MTCNN` (Multi-task Cascaded Convolutional Networks).
2.  **Classificador (Reconhecimento) de Faces:** Rede pré-treinada `FaceNet` para gerar *embeddings* (vetores de características) e um classificador baseado em distância para identificar a pessoa.

## 1. Setup do Ambiente

A célula abaixo contém os comandos de instalação necessários para executar este notebook. Se você já configurou seu ambiente virtual conforme o `README.md`, pode pular esta etapa.

In [ ]:
# Comandos para instalação via pip (execute no terminal dentro do seu ambiente virtual)
# !pip install tensorflow jupyterlab mtcnn keras-facenet opencv-python matplotlib scikit-learn

print("Ambiente pronto para execução.")

## 2. Configurações Globais

Centralizar as configurações aqui facilita a alteração de parâmetros sem precisar modificar o restante do código.

In [ ]:
# --- PARÂMETROS CONFIGURÁVEIS ---

# Diretório onde as imagens das pessoas conhecidas estão armazenadas
DATABASE_DIR = 'database'

# Caminho para a imagem de teste que será analisada
TEST_IMAGE_PATH = 'test_images/imagem_de_teste.jpg'

# Limiar de confiança para o reconhecimento. 
# Um valor menor torna o sistema mais rigoroso. Valores típicos ficam entre 0.4 e 0.7.
RECOGNITION_THRESHOLD = 0.5

# Tamanho da entrada da rede FaceNet
FACENET_INPUT_SIZE = (160, 160)

print("Configurações definidas.")

## 3. Importação das Bibliotecas

In [ ]:
import os

import cv2
import numpy as np
from keras_facenet import FaceNet
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import LabelEncoder

# Silenciar logs excessivos do TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("Bibliotecas importadas com sucesso.")

## 4. Inicialização dos Modelos

In [ ]:
# Inicializar os modelos pré-treinados
print("Carregando modelos... Isso pode levar alguns segundos.")

# Detector de Faces (MTCNN)
detector = MTCNN()

# Extrator de Embeddings (FaceNet)
embedder = FaceNet()

print("Detector (MTCNN) e Extrator de Embeddings (FaceNet) carregados.")

## 5. Construção do Banco de Dados de Faces

Nesta etapa, vamos processar as imagens do diretório `database`. Para cada rosto encontrado, extraímos seu ***embedding*** (uma representação vetorial de 512 dimensões, que funciona como uma "impressão digital" matemática) e o armazenamos.

In [ ]:
def load_face_database(directory):
    """
    Carrega imagens de um diretório, detecta faces, extrai embeddings
    e armazena junto com os nomes das pessoas.
    """
    database = {'embeddings': [], 'names': []}
    
    for person_name in os.listdir(directory):
        person_dir = os.path.join(directory, person_name)
        if not os.path.isdir(person_dir):
            continue
            
        print(f"Processando: {person_name}...")
        for filename in os.listdir(person_dir):
            path = os.path.join(person_dir, filename)
            
            image = cv2.imread(path)

            if image is None:
                print(f"  Aviso: Não foi possível carregar a imagem {path}. Pulando.")
                continue
            
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = detector.detect_faces(image_rgb)
            
            if results:
                best_result = max(results, key=lambda r: r['confidence'])
                x1, y1, width, height = best_result['box']
                x1, y1 = abs(x1), abs(y1)
                x2, y2 = x1 + width, y1 + height
                
                face = image_rgb[y1:y2, x1:x2]
                

                face_resized = cv2.resize(face, FACENET_INPUT_SIZE)
                face_array = np.asarray(face_resized)
                face_expanded = np.expand_dims(face_array, axis=0)
                
                embedding = embedder.embeddings(face_expanded)
                
                database['embeddings'].append(embedding[0])
                database['names'].append(person_name)

    database['embeddings'] = np.asarray(database['embeddings'])
    return database

# Carregar nosso banco de dados usando a variável de configuração
known_faces_db = load_face_database(DATABASE_DIR)

# Codificar os nomes para facilitar a manipulação
le = LabelEncoder()
labels = le.fit_transform(known_faces_db['names'])

print("\nBanco de dados de faces carregado com sucesso.")
print(f"Total de amostras de faces: {len(known_faces_db['names'])}")
print(f"Pessoas únicas no banco de dados: {len(np.unique(known_faces_db['names']))}")

## 6. Função Principal de Detecção e Reconhecimento

Esta função recebe uma nova imagem e realiza o processo de reconhecimento. Para cada rosto detectado, ela calcula seu *embedding* e o compara com os do banco de dados usando a **distância cosseno**. Se a distância for menor que o `RECOGNITION_THRESHOLD`, consideramos que a pessoa foi reconhecida.

In [ ]:
def recognize_faces(image_path, db, threshold):
    """
    Lê uma imagem, detecta todas as faces e tenta reconhecê-las
    comparando com o banco de dados.
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"Erro: Não foi possível carregar a imagem de teste em {image_path}")
        return None
        
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    detections = detector.detect_faces(image_rgb)
    
    if not detections:
        print("Nenhuma face detectada na imagem de teste.")
        return image
    
    print(f"Detectadas {len(detections)} faces.")
    
    for face_info in detections:
        x1, y1, width, height = face_info['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        
        face = image_rgb[y1:y2, x1:x2]
        if face.size == 0: continue
            
        face_resized = cv2.resize(face, FACENET_INPUT_SIZE)
        face_array = np.asarray(face_resized)
        face_expanded = np.expand_dims(face_array, axis=0)
        
        new_embedding = embedder.embeddings(face_expanded)[0]
        
        distances = pairwise_distances([new_embedding], db['embeddings'], metric='cosine')[0]
        
        min_dist_idx = np.argmin(distances)
        min_dist_val = distances[min_dist_idx]
        
        # Reconhecido
        if min_dist_val < threshold:
            person_name = le.inverse_transform([labels[min_dist_idx]])[0]
            # Remove o prefixo do nome da pasta para exibição
            display_name = person_name.replace('_', ' ').title()
            label = f"{display_name} ({min_dist_val:.2f})"
            color = (0, 255, 0) # Verde
        # Não reconhecido
        else:
            label = f"Desconhecido ({min_dist_val:.2f})"
            color = (0, 0, 255) # Vermelho

        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    return image

## 7. Execução e Visualização do Resultado

In [ ]:
# Executar o reconhecimento usando as variáveis de configuração
result_image = recognize_faces(TEST_IMAGE_PATH, known_faces_db, threshold=RECOGNITION_THRESHOLD)

# Exibir o resultado usando Matplotlib
if result_image is not None:
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
    plt.title("Resultado do Reconhecimento Facial")
    plt.axis('off')
    plt.show()

## 8. (Bônus) Reconhecimento em Tempo Real com Webcam

A célula abaixo ativa a webcam e aplica o mesmo processo de reconhecimento em tempo real. Pressione a tecla 'q' para fechar a janela.

In [ ]:
def run_realtime_recognition(db, threshold):
    """Ativa a webcam e executa o reconhecimento facial em tempo real."""
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Erro: Não foi possível abrir a webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # A lógica de reconhecimento é a mesma da função anterior, aplicada ao frame da webcam
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detections = detector.detect_faces(frame_rgb)
        
        for face_info in detections:
            x1, y1, width, height = face_info['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            
            face = frame_rgb[y1:y2, x1:x2]
            if face.size == 0: continue
            
            face_resized = cv2.resize(face, FACENET_INPUT_SIZE)
            face_array = np.asarray(face_resized)
            face_expanded = np.expand_dims(face_array, axis=0)
            
            new_embedding = embedder.embeddings(face_expanded)[0]
            
            distances = pairwise_distances([new_embedding], db['embeddings'], metric='cosine')[0]
            min_dist_idx = np.argmin(distances)
            min_dist_val = distances[min_dist_idx]
            
            if min_dist_val < threshold:
                person_name = le.inverse_transform([labels[min_dist_idx]])[0]
                display_name = person_name.replace('_', ' ').title()
                label = f"{display_name}"
                color = (0, 255, 0)
            else:
                label = "Desconhecido"
                color = (0, 0, 255)

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        cv2.imshow('Reconhecimento Facial - Pressione "q" para sair', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Para executar o modo webcam, descomente a linha abaixo:
# run_realtime_recognition(known_faces_db, threshold=RECOGNITION_THRESHOLD)